# Qoffee Maker

In [ ]:
import os, time
from random import randint
from dotenv import load_dotenv
import numpy as np
load_dotenv();

In [ ]:
from ibm_quantum_widgets import CircuitComposer
from qiskit.circuit import QuantumCircuit
from qiskit import transpile, IBMQ
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator
from qiskit import IBMQ
from qiskit.providers.ibmq.exceptions import IBMQAccountError

In [ ]:
try:
    IBMQ.enable_account(os.getenv("IBMQ_API_KEY"))
except IBMQAccountError as e:
    print("Got IBMAccountError", e, "probably due to re-execution")

provider = IBMQ.get_provider()
provider.backends()

In [ ]:
backend_ideal = StatevectorSimulator()
backend_noise = QasmSimulator()
backend_ibm = provider.get_backend("ibmq_manila")

In [ ]:
## build some sample circuits

# empty
qc0 = QuantumCircuit(3, 3)

# cappucino
qc1 = QuantumCircuit(3, 3)
qc1.x(2)

# coffee
qc2 = QuantumCircuit(3, 3)
qc2.x(0)
qc2.h(1)
qc2.z(1)
qc2.h(1)

# espresso
qc3 = QuantumCircuit(3, 3)
qc3.x(0)
qc3.cx(0, 1)
qc3.cx(1, 0)
qc3.cx(0, 1)

# anything
qc4 = QuantumCircuit(3, 3)
qc4.h(0)
qc4.h(1)
qc4.h(2)

# without coffein
qc5 = QuantumCircuit(3, 3)
qc5.h(0)

# nothing strong
qc6 = QuantumCircuit(3, 3)
qc6.h(0)
qc6.x(2)

# tea or coffee
qc7 = QuantumCircuit(3, 3)
qc7.h(0)
qc7.cx(0, 1);


In [ ]:
qcreg = [
    {
        "id": "empty",
        "name": "From Scratch",
        "circuit": qc0
    },
    {
        "id": "cappucino",
        "name": "Only Cappucino",
        "circuit": qc1
    },
    {
        "id": "coffee",
        "name": "Only Coffee",
        "circuit": qc2
    },
    {
        "id": "espresso",
        "name": "Only Espresso",
        "circuit": qc3
    },
    {
        "id": "anything",
        "name": "Just something to drink",
        "circuit": qc4
    },
    {
        "id": "withoutcoffein",
        "name": "Without Coffein",
        "circuit": qc5
    },
    {
        "id": "nothingstrong",
        "name": "Nothing Strong",
        "circuit": qc6
    },
    {
        "id": "teaorcoffee",
        "name": "Tea or Coffee",
        "circuit": qc7
    }
]

In [ ]:
class CircuitExecutor:
    
    BIT_ORDER = ['000', '001', '010', '011', '100', '101', '110', '111']
    
    def __init__(self, circuit):
        self.circuit = circuit
        
    def _fill_with_zero(results):
        for bit in CircuitExecutor.BIT_ORDER:
            if bit not in results:
                results[bit] = 0
        return results
    
    def get_maximum_key(results):
        max_key = max(results, key=results.get)
        return max_key
        
    def measure(self, num_shots=800):
        # copy current circuit
        mqc = self.circuit.copy()
        # add measurement
        mqc.measure(range(3), range(3))
        # transpile circuit --> necessary?
        # mqc_compiled = transpile(mqc, backend)
        # run job and get result
        result = backend_noise.run(mqc, shots=num_shots, seed=randint(0, 2500)).result()
        counts = result.get_counts(mqc)
        # divide by num_shots to get probabilities
        for b in counts:
            counts[b] = float(counts[b])/num_shots
        return CircuitExecutor._fill_with_zero(counts)
    
    def solve(self):
        mqc = self.circuit.copy()
        # run circuit on state vector
        result = backend_ideal.run(mqc).result()
        # get state vector and calculate probabilities
        state_vector = result.get_statevector(mqc, decimals=5)
        probs = np.abs(state_vector ** 2)
        # map back to bit configurations
        results = {
            key: probs[i] for i, key in enumerate(CircuitExecutor.BIT_ORDER)
        }
        return CircuitExecutor._fill_with_zero(results)
    
    def run_on_q(self, num_shots=1, secs_timeout=10):
        # copy current circuit
        mqc = self.circuit.copy()
        # add measurement
        mqc.measure(range(3), range(3))
        mqc_transpiled = transpile(mqc, backend_ibm)
        # run job and get result
        job = backend_ibm.run(mqc_transpiled, shots=num_shots)
        
        counts = None
        try:
            result = job.result(timeout=secs_timeout, wait=2)
            counts = result.get_counts(mqc_transpiled)
            # divide by num_shots to get probabilities
            for b in counts:
                counts[b] = float(counts[b])/num_shots
        except Exception as e:
            # did not get results, so try to get them from internal measure
            print("Got error from IBMQ", e)
            counts = self.measure(num_shots=num_shots)
        
        return CircuitExecutor._fill_with_zero(counts)

# Build App

In [ ]:
from ipywidgets import Widget, Button, HTML, VBox, widgets, link, HBox
import appwidgets
from traitlets import Unicode, Dict, Int, Any, HasTraits
from appwidgets import JsPyWidget
import json

## Build Data Model

In [ ]:
DRINKS = {
    "000": {
        "id": "tea",
        "name": "Tee",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.WarmMilk",
        "options": {}
    },
    "001": {
        "id": "hotchocolate",
        "name": "Heisse Schokolade",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.MilkFroth",
        "options": {}
    },
    "010": {
        "id": "espresso",
        "name": "Espresso",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.Espresso",
        "options": {}
    },
    "011": {
        "id": "coffee",
        "name": "Kaffee",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.Coffee",
        "options": {}
    },
    "100": {
        "id": "cappucino",
        "name": "Cappucino",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.Cappuccino",
        "options": {}
    },
    "101": {
        "id": "lattemacchiato",
        "name": "Latte Macchiato",
        "key": "ConsumerProducts.CoffeeMaker.Program.Beverage.LatteMacchiato",
        "options": {}
    },
    "110": {
        "id": "wienermelange",
        "name": "Wiener Melange",
        "key": "ConsumerProducts.CoffeeMaker.Program.CoffeeWorld.WienerMelange",
        "options": {}
    },
    "111": {
        "id": "americano",
        "name": "Americano",
        "key": "ConsumerProducts.CoffeeMaker.Program.CoffeeWorld.Americano",
        "options": {}
    }
}

In [ ]:
data = Widget()
data.add_traits(
    # name of current circuit with which the composer was initialized
    composer_init_circuit = Unicode("empty").tag(sync=True),
    # current ideal probabilities
    histogram_ideal = Dict({
        key: 0 for key in DRINKS
    }).tag(sync=True),
    # current measured probabilities
    histogram_measured = Dict({
        key: 0 for key in DRINKS
    }).tag(sync=True),
    # id of drink from single shot measurement
    single_shot_result = Dict({
        "name": "",
        "id": "",
        "bit": ""
    }).tag(sync=True),
    # status of single shot measurements (0 = init, 1 = loading, 2 = done)
    single_shot_status = Int().tag(sync=True),
    # device on which to perform the single shot
    single_shot_device = Unicode("simulator").tag(sync=True),
    # composer qasm
    qasm = Unicode("").tag(sync=True)
)

## Build Composer

In [ ]:
composer = CircuitComposer(circuit=qcreg[0]['circuit'])

In [ ]:
# handler for change events
def composer_update_handler(change_event):
    # there are multiple change events, only use on
    if change_event['name'] != "circuit":
        return
    executor = CircuitExecutor(composer.circuit)
    # get and save ideal result
    result_ideal = executor.solve()
    data.histogram_ideal = result_ideal
    # get and save measurement
    result_measured = executor.measure()
    data.histogram_measured = result_measured
    # get and save qasm code
    qasm = composer.circuit.qasm()
    data.qasm = qasm
    
composer.observe(composer_update_handler)

In [ ]:
# perform single shot measurement
def get_single_shot_result():
    data.single_shot_status = 1
    executor = CircuitExecutor(composer.circuit)
    counts = None
    if data.single_shot_device == "ibmq":
        counts = executor.run_on_q(num_shots=1, secs_timeout=60)
    else:
        counts = executor.measure(num_shots=1)
        
    bits = CircuitExecutor.get_maximum_key(counts)
    data.single_shot_result = {
        "bit": bits,
        "id": DRINKS[bits]["id"],
        "name": DRINKS[bits]["name"]
    }
    time.sleep(1.5)
    data.single_shot_status = 2
    return bits

In [ ]:
%%javascript
window.requestDrink = function(drinkKey, drinkOptions) {
    return new Promise((resolve, reject) => {
        // do POST request to Jupyter backend
        fetch("/drink", {
            method: 'post',
            credentials: 'same-origin',
            headers: {
                'X-XSRFToken': document.cookie.replace("_xsrf=", "")
            },
            body: JSON.stringify({
                key: drinkKey,
                options: drinkOptions
            })
        }).then(response => {
            // if fail, alert and go to welcome
            if(!response.ok) {
                alert("Could not get drink "+drinkKey+"\n"+response.statusText);
                reject();
            }
            // if succeed to to success
            else {
                resolve();
            }
        }, error => {
            alert("Could not get drink "+drinkKey+"\n"+error);
            console.error(error);
            reject(error);
        })
    })
}

In [ ]:
def request_drink():
    # get result from single shot measurement
    bit = data.single_shot_result['bit']
    drink_key = DRINKS[bit]['key']
    drink_options = DRINKS[bit]['options']
    
    def response_handler(res):
        if 'error' in res:
            app.view = 'error'
        else:
            app.view = 'success'
    
    jspy.execute_js(
        "window.requestDrink("+json.dumps(drink_key)+", "+json.dumps(drink_options)+")",
        response_handler
    )

## Build views

In [ ]:
app = appwidgets.AppBox(init_view='welcome')
jspy = JsPyWidget()

### Welcome View

In [ ]:
view1 = appwidgets.ReactiveHtmlWidget("""
    <h1>Welcome to Qoffee Maker</h1>
    <button class="btn btn-primary" data-rh-exec="app.view='startingpoints'">Get your coffee</button>
""")

### Starting Point View

In [ ]:
view2_header = HTML("<h1>Choose your starting point</h1>")
view2_elements = [view2_header]

def get_click_button_handler(cid, circuit):
    def click_button_handler(*args):
        data.composer_init_circuit = cid
        data.single_shot_device = 'simulator'
        data.single_shot_status = 0
        composer.circuit = circuit
        app.view = 'composer'
    return click_button_handler

for qc in qcreg:
    qc_button = Button(
        description=qc["name"]
    )
    qc_button.on_click(get_click_button_handler(qc["id"], qc["circuit"]))
    view2_elements.append(qc_button)

view2 = VBox(view2_elements)

### Composer View

In [ ]:
view3_header = HTML("""
    <h1>Build your quantum circuit</h1>
    <p>Drag and drop gates on the circuit to build the circuit</p>
""")


hist_html = ""
for key, value in DRINKS.items():
    hist_html += """
        <div class="histogram-entry">
            <div class="histogram-bar-container">
                <div class="histogram-bar-outer">
                    <div style="height: {{${histogram_ideal}['"""+key+"""']*100}}%" class="histogram-bar histogram-bar-ideal histogram-bar-"""+key+""""></div>
                    <div style="height: {{${histogram_measured}['"""+key+"""']*100}}%" class="histogram-bar histogram-bar-measured histogram-bar-"""+key+""""></div>
                </div>
            </div>
            <div class="histogram-text-container">
                <p>"""+value["name"]+"""<br/><span class="histogram-text histogram-text-ideal histogram-text-"""+key+""""">{{int(${histogram_measured}['"""+key+"""']*1000)/10}}%</span></p>
            </div>
        </div>
    """

view3_hist = appwidgets.ReactiveHtmlWidget("""
<div id="composer-histogram-container">
    <div id="composer-histogram">"""+hist_html+"""</div>
</div>
""", data_model=data)


view3_singleshot = appwidgets.ReactiveHtmlWidget("""
    <div id="composer-singleshot-container">
        <div id="composer-singleshot">
            <button class="btn btn-default btn-block" data-rh-set='single_shot_device="ibmq"' data-rh-if='${single_shot_device}=="simulator"'>Benutze IBM Q</button>
            <button class="btn btn-default btn-block" data-rh-set='single_shot_device="simulator"' data-rh-if='${single_shot_device}=="ibmq"'>Benutze Simulator</button>
            <br/>
            <button class="btn btn-primary btn-block" data-rh-exec="get_single_shot_result()">Messung vornehmen</button>
            <br/>
            <div data-rh-if='${single_shot_status}==1'>
                Loading...
            </div>
            <div data-rh-if='${single_shot_status}==2'>
                <p>Die Messung ergab: <b>{{${single_shot_result}["name"]}}</b>
                <button class="btn btn-primary btn-block" data-rh-exec="request_drink()">Okay</button>
            </div>
            <br/>
            <button class="btn btn-default btn-block" data-rh-exec-js="openQRCodeIBMQ()">In IBM Q öffnen</button>
            <button class="btn btn-default btn-block" data-rh-exec="app.view='startingpoints'">Zurück</button>
        </div>
    </div>
""", data_model=data)

view3_hist.load_css("./css/histogram.css")

In [ ]:
view3 = VBox([
    view3_header, 
    composer,
    HBox([
        view3_hist, view3_singleshot
    ])
])

In [ ]:
view4 = appwidgets.ReactiveHtmlWidget("""
    <h1>Oh no, something went wrong :/</h1>
""")


view5 = appwidgets.ReactiveHtmlWidget("""
    <h1>Thank you and enjoy your coffee :)</h1>
""")

In [ ]:
app.add_widget('welcome', view1)
app.add_widget('startingpoints', view2)
app.add_widget('composer', view3)
app.add_widget('error', view4)
app.add_widget('success', view5)

# And go...

In [ ]:
%%html
<style>
/*
	Remove checkbox from below editor
*/
.composer-workarea > div.duo--Checkbox-wrapper {
	display: none;
}
</style>

In [ ]:
app